In [ ]:
from qdrant_client import QdrantClient, models

In [ ]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [ ]:
model_name = 'snowflake/snowflake-arctic-embed-m-long'
emb_dimensions = 768

In [ ]:
collection_name = "bfp-a3447q_test"

if client.collection_exists(collection_name):
    client.get_collection(collection_name)
    print(f"Collection {collection_name} restored")
else:
    print(f"Collection {collection_name} not found.\nCreate collection first")
    
    

In [ ]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #query embedding
            text=query,
            model=model_name 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [ ]:
results = search("How to check if controller have been ready?", limit = 20)

In [ ]:
for index, payload in enumerate(results.points):
    print(f"{index}) Score: {payload.score}, Point id: {payload.id}\n\tMain Chapter:{payload.payload['main_chapter']}\n\tChapter:{payload.payload['chapter']}\nContent: {payload.payload['content'][:200]}")
    print("##############################################################\n")